In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
# import tensorflow.compat.v2 as tf
import keras
import os

In [ ]:
# # code_path=os.path.abspath(os.getcwd())
# # data_path = './Datasets/Gravity_waves/data' 
# # data_path = ('/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/data')
# # data_path = ('/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/new-sar-data')
data_path = ('/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png')
img_path= data_path
os.chdir(img_path)
print(os.path.abspath(os.getcwd()))

/content/drive/.shortcut-targets-by-id/1KS2muRwn4VXnctrNWa5dx0K8BIIRSELD/ESIP-Ocean-Eddy-Detection/demo_png


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot11.png', show_shapes=True, show_layer_names=True)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 32)       0

In [ ]:
batch_size = 16

from keras.preprocessing.image import ImageDataGenerator

# # this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False) ## cannot do flip

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
        rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/train',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        color_mode = 'grayscale',
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/validation',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode='binary')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/ESIP-Ocean-Eddy-Detection/demo_png/test',
        target_size=(256, 256),
        color_mode = 'grayscale',
        batch_size=1,
        class_mode=None,
        shuffle=False)

Found 42 images belonging to 2 classes.
Found 22 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=200 // batch_size,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=80 // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
12/12 [==============================] - 11s 736ms/step - loss: 0.4300 - accuracy: 0.6429 - val_loss: 0.0494 - val_accuracy: 1.0000


In [ ]:
nb_test_samples = 15
probabilities_temp = model.predict_generator(test_generator, nb_test_samples)

from sklearn.metrics import confusion_matrix
from scipy.special import expit

probabilities = expit(probabilities_temp)

y_true = np.array([0] * 13 + [1] * 2)
y_pred = probabilities

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_true, y_pred)
average_precision

1.0